<a href="https://colab.research.google.com/github/Takumi173/JPMA2023TF1-1/blob/main/JPMA2023_case1_(1)_%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%8B%A1%E5%BC%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続とパスの指定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 拡張データの保存先ディレクトリの指定（ディレクトリは事前に用意する）
datadir = '/content/drive/MyDrive/JPMA2023_case1/'

Mounted at /content/drive


## ライブラリとパッケージのインストール

In [2]:
!pip install mecab-python3
!pip install unidic-lite
!export MECABRC='/etc/mecabrc'

import pandas as pd
import random
import MeCab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=a7450d0019230d2df6dde3ff70d1cd2825ad5a5c5ebe4b5572221b23de5801a6
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


## データの読み込み

In [3]:
# ICD-10の３桁分類データおよび再翻訳データのパス（U,V,Y,Zカテゴリを除く）
Training_data   = 'https://github.com/Takumi173/JPMA2023TF1-1/releases/download/20240529/Case1_ICD10_data.tsv'
Translated_data = 'https://github.com/Takumi173/JPMA2023TF1-1/releases/download/20240529/Case1_Retranslated_data.tsv'

In [4]:
# Loading training data
df = pd.read_csv(Training_data, sep = "\t", header=0, encoding = "utf-8",
                # 3 digit
                dtype = {"ICD10_CATEGORY": "str", "ICD10_Code": "str", "ICD10_Term": "str", "FOR_USAGE" : "str"})
df

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラ菌によるコレラ,train
1,A00,コレラ,A00.1,エルトールコレラ菌によるコレラ,train
2,A00,コレラ,A00.9,コレラ，詳細不明,train
3,A01,腸チフス及びパラチフス,A01.0,腸チフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラチフスA,train
...,...,...,...,...,...
10524,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,鋭利な物体による加害にもとづく傷害及び死亡；商業及びサービス施設,train
10525,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,鋭利な物体による加害にもとづく傷害及び死亡；工業用地域及び建築現場,train
10526,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,鋭利な物体による加害にもとづく傷害及び死亡；農場,train
10527,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,鋭利な物体による加害にもとづく傷害及び死亡；その他の明示された場所,train


In [5]:
# Loading translated data
df_translate = pd.read_csv(Translated_data, sep = "\t", header=0, encoding = "utf-8",
                # 3 digit
                dtype = {"ICD10_CATEGORY": "str", "ICD10_CATEGORY_TERM": "str", "ICD10_Code": "str", "ICD10_Term": "str", "FOR_USAGE" : "str"})
df_translate

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラ菌によるコレラ,train
1,A00,コレラ,A00.1,エルトールコレラによるコレラ,train
2,A00,コレラ,A00.9,コレラ、詳細不明,train
3,A01,腸チフス及びパラチフス,A01.0,腸チフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラチフスA,train
...,...,...,...,...,...
10524,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,鋭利な物体による傷害による死傷者；商業・サービス施設,train
10525,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,鋭利な物体による傷害による死傷者；工業地域および建設現場,train
10526,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,鋭利な物体による傷害による死傷者；農場,train
10527,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,鋭利な物体による負傷による死傷者；その他の指定場所,train


# データ拡張

## ランダムシードの指定

In [6]:
# ランダム生成を固定する（任意の数字を指定）
random_seed = 56789

## 拡張データの保存ファイル名指定

In [7]:
# 助詞追加
Output_postposition_tsv = 'postposition.tsv'

# ランダムな位置に１文字のランダムなひらがなを挿入、
Output_rand_ins_hiragana_tsv = 'rand_ins_hiragana.tsv'

# 同義語変換
Output_synonym_tsv = 'synonym.tsv'

# 転置
Output_reverse_tsv = 'reverse.tsv'

# データ拡張した最終ファイル
Output_combined_training_data_tsv = 'combined_training_data_(Seed_' + str(random_seed) + ').tsv'
Output_combined_training_data_tsv

'combined_training_data_(Seed_56789).tsv'

## Case1: 助詞追加

In [8]:
df_postposition= df.copy()

add_texts = [
    ['pre', 'あの'],
    ['pre', 'いつもの'],
    ['post', 'で'],
    ['post', 'により'],
    ['post', 'のため'],
    ['post', 'から'],
    ['post', 'の悪化'],
    ['post', 'が悪化'],
    ['both', '(',')']
]

# ループを実行して文字列を追加し、新しいDataFrameを作成
for text in add_texts:
    # 新しいDataFrameを作成し文字列を追加
    df_tmp = df.copy()

    if text[0] == 'pre':
      df_tmp["ICD10_Term"] = text[1] + df_tmp["ICD10_Term"]

    if text[0] == 'post':
      df_tmp["ICD10_Term"] = df_tmp["ICD10_Term"] + text[1]

    if text[0] == 'both':
      df_tmp["ICD10_Term"] = text[1] + df_tmp["ICD10_Term"] + text[2]

    # 追加結果の確認
    print(df_tmp[0:1]["ICD10_Term"].tolist()[0])

    # 新しいDataFrameを元のDataFrameに結合
    df_postposition = pd.concat([df_postposition, df_tmp], ignore_index=True)

# 結果の保存と表示
df_postposition.to_csv(Output_postposition_tsv, sep='\t', index=False)
df_postposition

あのコレラ菌によるコレラ
いつものコレラ菌によるコレラ
コレラ菌によるコレラで
コレラ菌によるコレラにより
コレラ菌によるコレラのため
コレラ菌によるコレラから
コレラ菌によるコレラの悪化
コレラ菌によるコレラが悪化
(コレラ菌によるコレラ)


,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラ菌によるコレラ,train
1,A00,コレラ,A00.1,エルトールコレラ菌によるコレラ,train
2,A00,コレラ,A00.9,コレラ，詳細不明,train
3,A01,腸チフス及びパラチフス,A01.0,腸チフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラチフスA,train
...,...,...,...,...,...
105285,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,(鋭利な物体による加害にもとづく傷害及び死亡；商業及びサービス施設),train
105286,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,(鋭利な物体による加害にもとづく傷害及び死亡；工業用地域及び建築現場),train
105287,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,(鋭利な物体による加害にもとづく傷害及び死亡；農場),train
105288,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,(鋭利な物体による加害にもとづく傷害及び死亡；その他の明示された場所),train


## Case2: ランダムな位置にひらがなを挿入

In [9]:
# 挿入するひらがなの文字数を指定
rand_char = 1

# ランダムなひらがな文字列作成する関数
def create_hiragana(rand_char):
    hiragana = ""

    # ひらがなのUnicodeコードポイントの範囲を指定して生成: U+3041 から U+3093
    for i in range(rand_char):
      # random_seedのグローバル変数宣言
      global random_seed

      random.seed(random_seed)
      hiragana += chr(random.randint(0x3041, 0x3093))

    return hiragana

# ランダムな位置に作成したひらがなを挿入する関数
def ins_rand_hiragana(text):
    # random_seedのグローバル変数宣言
    global random_seed

    #挿入位置をランダムに選択
    random.seed(random_seed)
    insert_index = random.randint(0, len(text))

    #ひらがな文字列を作成
    random_hiragana = create_hiragana(rand_char)

    #ひらがなを挿入
    modified_text = text[:insert_index] + random_hiragana + text[insert_index:]

    return modified_text

# dfをコピーして関数実行
df_rand_ins_hiragana = df.copy()
df_rand_ins_hiragana['ICD10_Term'] = df_rand_ins_hiragana['ICD10_Term'].apply(ins_rand_hiragana)

# 結果の保存と表示
df_rand_ins_hiragana.to_csv(Output_rand_ins_hiragana_tsv, sep='\t', index=False)
df_rand_ins_hiragana

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラ菌てによるコレラ,train
1,A00,コレラ,A00.1,エルトールコレラ菌てによるコレラ,train
2,A00,コレラ,A00.9,コレラ，て詳細不明,train
3,A01,腸チフス及びパラチフス,A01.0,腸チてフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラてチフスA,train
...,...,...,...,...,...
10524,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,鋭利な物体による加害にもとづく傷害及てび死亡；商業及びサービス施設,train
10525,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,鋭利な物体による加害にもとづく傷害及てび死亡；工業用地域及び建築現場,train
10526,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,鋭利な物体による加て害にもとづく傷害及び死亡；農場,train
10527,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,鋭利な物体による加害にもとづく傷害及てび死亡；その他の明示された場所,train


## Case3: 同義語変換

In [10]:
# MeCabの初期化
mecab = MeCab.Tagger()

# dfをコピーして、新規dfを作成
df_synonym = df.copy()

# 同義語辞書を使用しテキスト置換用の関数
def replace_synonyms(text):
  # random_seedのグローバル変数宣言
  global random_seed

  new_words = []

  for word in mecab.parse(text).split("\n"):
    words = word.split("\t")[:-4]
    if len(words) -1 <= 0:
      new_words.append("")
    else:
      random.seed(random_seed)
      rand_index = random.randint(0, len(words)-1)
      new_words.append(words[rand_index])
      random_seed += 1  # 取得する位置が変わるようにランダムシードを単語ごとに変える

  new_text = "".join(new_words).strip()
  return new_text

# dfをコピーして関数実行
df_synonym = df.copy()
df_synonym['ICD10_Term'] = df_synonym['ICD10_Term'].apply(replace_synonyms)

# 結果の保存と表示
df_synonym.to_csv(Output_synonym_tsv, sep='\t', index=False)
df_synonym

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラキンニヨルコレラ,train
1,A00,コレラ,A00.1,エルトールコレラ菌にヨルコレラ,train
2,A00,コレラ,A00.9,コレラ-cholera，ショーサイフメー,train
3,A01,腸チフス及びパラチフス,A01.0,腸チフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラチフスA,train
...,...,...,...,...,...
10524,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,鋭利だブッタイニよる加害にモトヅクショーガイ及び死亡ショーギョーオヨビサービスシセツ,train
10525,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,鋭利ダブッタイニヨル加害にモトヅク傷害及び死亡；工業用チイキオヨビケンチクゲンバ,train
10526,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,エイリなブッタイによるカガイニモトヅク傷害及びシボー農場,train
10527,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,エイリダブッタイに因る加害ニモトズクショーガイオヨビ死亡ソノ他ノ明示為るれたバショ,train


## Case4: 転置

In [11]:
# MeCabを使用してテキストを単語に分割する関数
def tokenize_japanese_text(text):
    mecab = MeCab.Tagger("-Owakati")  # MeCabの単語分割モードを指定
    words = mecab.parse(text).split()  # テキストを単語に分割
    return words

# テキストを転置する関数
def reverse_japanese_text(text):
    words = tokenize_japanese_text(text)  # テキストを単語に分割
    reversed_text = "".join(reversed(words))  # 単語を逆順に配置
    return reversed_text


# dfをコピーして関数実行
df_reverse = df.copy()
df_reverse['ICD10_Term'] = df_reverse['ICD10_Term'].apply(reverse_japanese_text)

# 結果の保存と表示
df_reverse.to_csv(Output_reverse_tsv, sep='\t', index=False)
df_reverse

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラよるに菌コレラ,train
1,A00,コレラ,A00.1,コレラよるに菌エルトールコレラ,train
2,A00,コレラ,A00.9,不明詳細，コレラ,train
3,A01,腸チフス及びパラチフス,A01.0,チフス腸,train
4,A01,腸チフス及びパラチフス,A01.1,Aパラチフス,train
...,...,...,...,...,...
10524,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,施設サービス及び商業；死亡及び傷害もとづくに加害よるに物体な鋭利,train
10525,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,現場建築及び地域用工業；死亡及び傷害もとづくに加害よるに物体な鋭利,train
10526,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,農場；死亡及び傷害もとづくに加害よるに物体な鋭利,train
10527,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,場所たれさ明示の他その；死亡及び傷害もとづくに加害よるに物体な鋭利,train


## 加工データの統合

In [12]:
# 翻訳を含む全てのdfを統合
df_combined_training_data= pd.concat([df_postposition, df_rand_ins_hiragana, df_synonym, df_reverse, df_translate],ignore_index=True)

# 結果をtsv出力+表示
df_combined_training_data.to_csv(Output_combined_training_data_tsv, sep='\t', index=False)
df_combined_training_data.to_csv(datadir + Output_combined_training_data_tsv, sep='\t', index=False)
df_combined_training_data

,ICD10_CATEGORY,ICD10_CATEGORY_TERM,ICD10_Code,ICD10_Term,FOR_USAGE
0,A00,コレラ,A00.0,コレラ菌によるコレラ,train
1,A00,コレラ,A00.1,エルトールコレラ菌によるコレラ,train
2,A00,コレラ,A00.9,コレラ，詳細不明,train
3,A01,腸チフス及びパラチフス,A01.0,腸チフス,train
4,A01,腸チフス及びパラチフス,A01.1,パラチフスA,train
...,...,...,...,...,...
147401,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X5,鋭利な物体による傷害による死傷者；商業・サービス施設,train
147402,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X6,鋭利な物体による傷害による死傷者；工業地域および建設現場,train
147403,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X7,鋭利な物体による傷害による死傷者；農場,train
147404,X99,鋭利な物体による加害にもとづく傷害及び死亡,X99.X8,鋭利な物体による負傷による死傷者；その他の指定場所,train
